In [ ]:
import sqlite3 as sql
import yfinance as yf
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from pandas_datareader import data as web 


import os
notebook_directory = os.getcwd()
print(notebook_directory)


# DATAFRAME PREPARATION

In [ ]:
stock_data = yf.download(tickers="IMX-USD", period = 'max', interval = '1d')


stock_data['CRYPTOCURRENCY_ID'] = 49

stock_data.reset_index(inplace=True)
stock_data['Date']  = pd.to_datetime(stock_data['Date'])
column_mapping = {
    'Date': 'TIMESTAMP',
    'Adj Close': 'PRICE',  
    'Volume': 'VOLUME'  
}
stock_data.rename(columns=column_mapping, inplace=True)
stock_data.drop(columns=['Open'], inplace=True)
stock_data.drop(columns=['High'], inplace=True)
stock_data.drop(columns=['Low'], inplace=True)
stock_data.drop(columns=['Close'], inplace=True)
stock_data.reset_index(drop=True, inplace=True)
stock_data

# INSERT DATAFRAME

In [ ]:
stock_data.reset_index(drop=True, inplace=True)


conn = sql.connect('/Users/lime/src/data-analytics/data/Crypto.db')

column_mapping = {
    'TIMESTAMP': 'TIMESTAMP',
    'PRICE': 'PRICE',
    'VOLUME': 'VOLUME',
    'CRYPTOCURRENCY_ID': 'CRYPTOCURRENCY_ID'
}

stock_data.rename(columns=column_mapping).to_sql('HISTORICAL_PRICES', conn, if_exists='append', index=False)

conn.commit()
conn.close()

# TABLE DEFINITION

In [ ]:
create_crp_table = '''CREATE TABLE IF NOT EXISTS CRYPTOCURRENCIES
    (ID INTEGER PRIMARY KEY AUTOINCREMENT,
    TICKER TEXT NOT NULL,
    NAME TEXT NOT NULL,
    DESCRIPTION TEXT,
    UNIQUE (ticker)
)'''


create_hp_table = '''CREATE TABLE IF NOT EXISTS HISTORICAL_PRICES
    (ID INTEGER PRIMARY KEY AUTOINCREMENT,
    CRYPTOCURRENCY_ID INTEGER,
    TIMESTAMP DATETIME,
    PRICE DECIMAL(10, 10),
    VOLUME DECIMAL(10, 10),
    FOREIGN KEY (CRYPTOCURRENCY_ID) REFERENCES CRYPTOCURRENCIES(ID)
)'''

create_index = '''CREATE INDEX IF NOT EXISTS idx_timestamp ON HISTORICAL_PRICES (TIMESTAMP)'''


In [ ]:
conn = sql.connect('/Users/lime/src/data-analytics/data/Crypto.db')
conn.execute(create_crp_table)
conn.execute(create_hp_table)
conn.execute(create_index)
conn.commit()
conn.close()

In [ ]:
conn = sql.connect('/Users/lime/src/data-analytics/data/test.db')

conn.execute("INSERT INTO CRYPTOCURRENCIES (SYMBOL, NAME, DESCRIPTION) VALUES ('BTC', 'Bitcoin', 'Network')")


conn.commit()
conn.close()

# DELETE database

In [ ]:

conn = sql.connect('/Users/lime/src/data-analytics/data/Crypto.db')

conn.execute("DROP TABLE IF EXISTS HISTORICAL_PRICES")


conn.commit()

conn.close()

# READ TABLE

In [ ]:

conn = sql.connect('/Users/lime/src/data-analytics/data/Crypto.db')

df_sum = pd.read_sql_query("SELECT * FROM TOKENSINFO", conn)


conn.commit()

conn.close()

# RENAME COLUMNS IN DATAFRAME

In [ ]:
df_sum.reset_index(drop=True, inplace=True)


column_mapping = {
    'Information': 'DESCRIPTION',
    'Ticker' : 'TICKER'
}

df_sum.rename(columns=column_mapping, inplace=True)

# SELECT A TABLE FROM DB

In [ ]:
conn = sql.connect('/Users/lime/src/data-analytics/data/Crypto.db')


query = '''SELECT * FROM HISTORICAL_PRICES WHERE CRYPTOCURRENCY_ID = 2'''

df = pd.read_sql_query(query, conn)

conn.close()
df

# RENAME TABLE

In [ ]:
conn = sql.connect('/Users/lime/src/data-analytics/data/Crypto.db')
cursor = conn.cursor()


cursor.execute('CREATE TABLE HISTORICAL_PRICES_1D AS SELECT * FROM HISTORICAL_PRICES;')

cursor.execute('DROP TABLE HISTORICAL_PRICES;')


conn.commit()
conn.close()

# LOAD THE DATA FROM SQL DB 

In [ ]:
conn = sql.connect('/Users/lime/src/data-analytics/data/Crypto.db')


query = '''SELECT TICKER FROM CRYPTOCURRENCIES '''

df = pd.read_sql_query(query, conn)

conn.close()
df = df.rename(index=lambda x: x + 1)
df

# LOAD THE DATA FROM SQL DB AND SAVE EACH TICKER IN SEPARETE DF

In [17]:
conn = sql.connect('/Users/lime/src/data-analytics/data/Crypto.db')

for index, row in df.iterrows():
    ticker = row['TICKER']

    query = f'''SELECT * FROM HISTORICAL_PRICES_1D WHERE CRYPTOCURRENCY_ID = {index}'''
    
    globals()[f'query_dict_{ticker.lower()}'] = pd.read_sql_query(query, conn)

conn.close()